In [ ]:
import torch
from torch.autograd import Variable
from torch.nn import functional as F
from torch.nn import Module

from stable_baselines3 import DDPG, SAC, TD3, DQN, PPO
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.utils import polyak_update

from tqdm import tqdm
import numpy as np

import os, shutil, sys
sys.path.append(os.path.abspath('./env'))
sys.path.append(os.path.abspath('./common'))

from env.env_simple_move import HumanMoveSimpleAction
from env.env_move_simple_v2 import MoveSimpleActionV2
from env.env_move_simple_v3 import MoveSimpleActionV3
from env.env_move_sector import HumanMoveSectorAction
from common.nets import MoveNet

from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [6]:
env = MoveSimpleActionV3()
n_state = env.observation_space.shape[0]
n_action = env.action_space.shape[0]
n_hidden = 256
lr = 0.005

In [49]:
env = HumanMoveSectorAction(object_ignore=True)
n_state = env.observation_space.shape[0]
n_action = env.action_space.shape[0]
n_hidden = 256
lr = 0.003


In [7]:
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
     
model = TD3('MlpPolicy', env, action_noise=action_noise, device=DEVICE)

In [50]:
model = DDPG('MlpPolicy', env, device=DEVICE)

In [ ]:
model.policy

In [10]:
def update(model, state, action, q_v):
    criterion = torch.nn.MSELoss()
    state = torch.tensor(state, dtype=torch.float32, device=DEVICE)
    action = torch.tensor(action, dtype=torch.float32, device=DEVICE)
    q_v = torch.tensor([q_v], dtype=torch.float32, device=DEVICE)

    action_by_net = model.actor.mu.forward(state)
    loss = criterion(action_by_net, action)

    model.actor.optimizer.zero_grad()
    loss.backward()
    model.actor.optimizer.step()

    critic_loss = None
    qvalue_input = torch.cat([state, action], dim=0)
    q_by_net = tuple(q_net(qvalue_input) for q_net in model.critic.q_networks)
    critic_loss = sum(criterion(current_q, q_v) for current_q in q_by_net)

    model.critic.optimizer.zero_grad()
    critic_loss.backward()
    model.critic.optimizer.step()

    polyak_update(model.actor.parameters(), model.actor_target.parameters(), model.tau)
    polyak_update(model.critic.parameters(), model.critic_target.parameters(), model.tau)



def get_action(model, state):
    state = torch.tensor(state, dtype=torch.float32, device=DEVICE).unsqueeze(0)
    action = model.actor.mu.forward(state)
    action = action.detach().cpu().numpy()[0]
    action = np.clip(action, -1.0, 1.0)
    return action

Обучение с учителем

In [ ]:
n_episode = 10000
total_reward_episode = np.zeros(n_episode)

for n in tqdm(range(n_episode)):
    state, _ = env.reset()
    action = get_action(model, state)
    teach_action = env.teach_action()
    next_state, reward, is_done, is_break, _ = env.step(teach_action)
    update(model, state, teach_action, reward)

Проверка модели

In [ ]:
n_episode = 10
total_reward_episode = np.zeros(n_episode)

for n in tqdm(range(n_episode)):

    state, _ = env.reset()
    
    is_done = False
    is_break = False
    sum_reward = 0

    while not is_done and not is_break:
        #action = model.get_action(state)
        action, _ = model.predict(state)
        next_state, reward, is_done, is_break, _ = env.step(action)
        sum_reward += reward
        state = next_state

    total_reward_episode[n] = sum_reward

In [ ]:
import matplotlib.pyplot as plt

plt.plot(total_reward_episode)
plt.title('Зависимость вознаграждения в эпизоде от времени')
plt.xlabel('Эпизод')
plt.ylabel('Полное вознаграждение')
plt.show()

In [ ]:
seed = int( datetime.now(TZ).strftime("%H%M%S") )
#env_render = HumanMoveSimpleAction(seed=seed, target_point_rand=True, render_mode = 'human')
env_render = HumanMoveSectorAction(seed=seed, target_point_rand=True, object_ignore=True, render_mode = 'human')
total_reward = 0.
step_reward = []
observation, _ = env_render.reset()
terminated = False
truncated = False
while not terminated and not truncated:
    #action = model.get_action(observation)
    action, _ = model.predict(observation)

    observation, reward, terminated, truncated, _ = env_render.step(action)
    total_reward += reward
    step_reward.append(reward)

total_reward

In [47]:
env_render.close()

In [ ]:
model.get_parameters()

In [39]:
torch.save(model.actor_target.state_dict(), './teached/move_simple_v3_TD3.pth')

In [40]:
t_model = torch.load('./teached/move_simple_v3_TD3.pth')

In [41]:
sample = torch.rand(3)

In [ ]:
sample

In [ ]:

scr_model = torch.jit.trace(model.policy, example_inputs=sample)

In [ ]:
scr_model = torch.jit.script(model.get_parameters())

In [ ]:
torch.jit.save(m=scr_model,f='./teached/script_move_simple_v3_TD3.pt')

In [ ]:
scr_model

In [14]:
model.save('./teached/move_simple_v3_TD3')

In [48]:
model.save('./teached/sector_move_TD3')

In [56]:
model.save('./teached/sector_move_DDPG')

In [36]:
read_model = TD3.load('./teached/simple_move_TD3', device='cuda')

In [2]:
read_model = TD3.load('./364/DDPG/model', device='cuda')

In [ ]:
#env_render = HumanMoveSimpleAction(render_mode = 'human',target_point_rand=True)
seed = int( datetime.now(TZ).strftime("%H%M%S") )
env_render = HumanMoveSectorAction(seed=seed, target_point_rand=False, object_ignore=True, render_mode = 'human')
total_reward = 0.
step_reward = []
observation, _ = env_render.reset(seed=seed)
terminated = False
truncated = False
while not terminated and not truncated:
    action, _ = read_model.predict(observation)
    observation, reward, terminated, truncated, _ = env_render.step(action)
    total_reward += reward
    step_reward.append(reward)

total_reward

In [6]:
env_render.close()